In [2]:
import pandas as pd
import glob

path = r'./journey_data'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    dfi = pd.read_csv(filename, index_col=None, header=0)
    li.append(dfi)

df = pd.concat(li, axis=0, ignore_index=True)




In [3]:
print(df.shape)

(3787948, 8)


In [4]:
df.describe()

,Departure station id,Return station id,Covered distance (m),Duration (sec.)
count,3.787948e+06,3.787889e+06,3.787948e+06,3.787948e+06
mean,1.666235e+02,1.665142e+02,2.084507e+03,9.375337e+02
std,2.001135e+02,2.002922e+02,3.598643e+03,8.144576e+03
min,1.000000e+00,1.000000e+00,-4.290436e+06,0.000000e+00
25%,3.600000e+01,3.500000e+01,9.650000e+02,3.270000e+02
50%,8.600000e+01,8.600000e+01,1.668000e+03,5.560000e+02
75%,2.020000e+02,2.020000e+02,2.779000e+03,9.320000e+02
max,9.980000e+02,9.980000e+02,3.589426e+06,5.401659e+06


In [5]:
df.head()

,Departure,Return,Departure station id,Departure station name,Return station id,Return station name,Covered distance (m),Duration (sec.)
0,2019-04-30T23:59:35,2019-05-01T00:00:36,43,Karhupuisto,43.0,Karhupuisto,2.0,57
1,2019-04-30T23:59:31,2019-05-01T00:09:00,140,Verkatehtaanpuisto,134.0,Haukilahdenkatu,2196.0,569
2,2019-04-30T23:59:25,2019-04-30T23:59:46,121,Vilhonvuorenkatu,121.0,Vilhonvuorenkatu,0.0,20
3,2019-04-30T23:59:21,2019-05-01T00:09:20,39,Ooppera,44.0,Sörnäisten metroasema,2121.0,596
4,2019-04-30T23:59:19,2019-05-01T00:18:12,57,Lauttasaaren ostoskeskus,63.0,Jätkäsaarenlaituri,2460.0,1127


In [6]:
collumns_to_drop = ["Departure station name", "Return station name", "Covered distance (m)", "Duration (sec.)"]
df.drop(collumns_to_drop, inplace=True, axis=1)
df.head()

,Departure,Return,Departure station id,Return station id
0,2019-04-30T23:59:35,2019-05-01T00:00:36,43,43.0
1,2019-04-30T23:59:31,2019-05-01T00:09:00,140,134.0
2,2019-04-30T23:59:25,2019-04-30T23:59:46,121,121.0
3,2019-04-30T23:59:21,2019-05-01T00:09:20,39,44.0
4,2019-04-30T23:59:19,2019-05-01T00:18:12,57,63.0


In [7]:
df['Departure'] = pd.to_datetime(df['Departure'])
df['Return'] = pd.to_datetime(df['Return'])
df.head()

,Departure,Return,Departure station id,Return station id
0,2019-04-30 23:59:35,2019-05-01 00:00:36,43,43.0
1,2019-04-30 23:59:31,2019-05-01 00:09:00,140,134.0
2,2019-04-30 23:59:25,2019-04-30 23:59:46,121,121.0
3,2019-04-30 23:59:21,2019-05-01 00:09:20,39,44.0
4,2019-04-30 23:59:19,2019-05-01 00:18:12,57,63.0


This data should be modified to match the format in the station data. So it should have collumns: 
"id", "bikes in", "bikes out", and "time", 
and each such row should contain the bikes coming in and going out in that five minute window, starting at "time".

Now here a problem: this data is accurate to the second, but the station data is collected every five minutes, and because of network lag (probably), it is usually timestamped 2 or 3 second after the full minute. And the timestaps are not uniform. So how do we allocate the journeys to blocks? Do we go through each block (id and time) in the station data, and allocate journeys according to the actual timestamp there?

In [12]:
arrivals = df.copy(deep=True)
arrivals.drop(["Departure", "Departure station id"], inplace=True, axis=1)
arrivals.columns=["time", "id"]
arrivals.head()

,time,id
0,2019-05-01 00:00:36,43.0
1,2019-05-01 00:09:00,134.0
2,2019-04-30 23:59:46,121.0
3,2019-05-01 00:09:20,44.0
4,2019-05-01 00:18:12,63.0


In [8]:
departures = df.copy(deep=True)
departures.drop(["Return", "Return station id"], inplace=True, axis=1)
departures.columns=["time", "id"]
departures.head(30)

,time,id
0,2019-04-30 23:59:35,43
1,2019-04-30 23:59:31,140
2,2019-04-30 23:59:25,121
3,2019-04-30 23:59:21,39
4,2019-04-30 23:59:19,57
5,2019-04-30 23:59:19,505
6,2019-04-30 23:59:14,647
7,2019-04-30 23:59:13,36
8,2019-04-30 23:59:09,225
9,2019-04-30 23:59:04,12


In [9]:
departures["time"]=departures["time"].dt.floor("5min")
departures.head(40)


,time,id
0,2019-04-30 23:55:00,43
1,2019-04-30 23:55:00,140
2,2019-04-30 23:55:00,121
3,2019-04-30 23:55:00,39
4,2019-04-30 23:55:00,57
5,2019-04-30 23:55:00,505
6,2019-04-30 23:55:00,647
7,2019-04-30 23:55:00,36
8,2019-04-30 23:55:00,225
9,2019-04-30 23:55:00,12


In [56]:
print(departures.shape)

(3787948, 2)


In [57]:
print(departures[(departures['time'] == '2019-06-30 23:50:00') & (departures['id'] == 707) ])

                  time   id
31 2019-06-30 23:50:00  707
32 2019-06-30 23:50:00  707
33 2019-06-30 23:50:00  707
34 2019-06-30 23:50:00  707
35 2019-06-30 23:50:00  707


In [10]:
departures['departures'] = 1
departures=departures.groupby(['time', 'id']).departures.count().reset_index()
departures.drop_duplicates()
print(departures.shape)
departures.head(40)

(2597579, 3)


,time,id,departures
0,2019-04-01 05:40:00,60,1
1,2019-04-01 05:50:00,85,1
2,2019-04-01 06:00:00,86,1
3,2019-04-01 06:05:00,26,1
4,2019-04-01 06:10:00,35,1
5,2019-04-01 06:15:00,30,1
6,2019-04-01 06:20:00,21,1
7,2019-04-01 06:30:00,30,1
8,2019-04-01 06:30:00,98,1
9,2019-04-01 06:30:00,731,1


In [59]:
print(departures[(departures['time'] == '2019-06-30 23:50:00') & (departures['id'] == 707) ])

                       time   id  departures
1157664 2019-06-30 23:50:00  707           5


In [62]:
print(departures[(departures['departures'] >50) ])

                      time  id  departures
189231 2019-04-23 20:30:00  76          52


In [13]:
arrivals["time"]=arrivals["time"].dt.floor("5min")
arrivals['arrivals'] = 1
arrivals=arrivals.groupby(['time', 'id']).arrivals.count().reset_index()
arrivals.drop_duplicates()


,time,id,arrivals
0,2019-04-01 05:50:00,25.0,1
1,2019-04-01 05:55:00,83.0,1
2,2019-04-01 06:10:00,64.0,1
3,2019-04-01 06:15:00,114.0,1
4,2019-04-01 06:30:00,30.0,1
...,...,...,...
2652353,2019-11-03 15:50:00,260.0,1
2652354,2019-11-03 16:55:00,731.0,1
2652355,2019-11-04 06:55:00,551.0,1
2652356,2019-11-04 10:30:00,138.0,2


In [20]:
print(arrivals[(arrivals['arrivals'] >20) ])

                       time     id  arrivals
111968  2019-04-16 12:30:00  203.0        24
296440  2019-05-01 12:20:00    1.0        22
296568  2019-05-01 12:30:00    1.0        21
296964  2019-05-01 13:00:00    1.0        23
1870510 2019-08-16 08:20:00   30.0        30
2010380 2019-08-26 08:35:00   30.0        23
2135339 2019-09-03 10:45:00  729.0        21
2147679 2019-09-04 07:15:00  547.0        22
2164692 2019-09-05 08:50:00   30.0        22
2287397 2019-09-16 10:35:00  204.0        21
2584571 2019-10-21 07:30:00   44.0        22


In [14]:
print(arrivals[(arrivals['id'] == 204) ])

                       time     id  arrivals
1112470 2019-06-26 13:50:00  204.0         2
1114534 2019-06-26 16:00:00  204.0         1
1115557 2019-06-26 16:40:00  204.0         2
1116479 2019-06-26 17:15:00  204.0         1
1118731 2019-06-26 18:50:00  204.0         2
...                     ...    ...       ...
2651201 2019-10-31 18:45:00  204.0         1
2651296 2019-10-31 19:00:00  204.0         1
2651318 2019-10-31 19:05:00  204.0         1
2651344 2019-10-31 19:10:00  204.0         1
2651515 2019-10-31 19:40:00  204.0         1

[4075 rows x 3 columns]


In [19]:
print(df[(df['Return station id'] == 204) ])

                  Departure              Return  Departure station id  \
407705  2019-06-30 22:03:49 2019-06-30 22:14:52                   112   
408025  2019-06-30 21:38:07 2019-06-30 21:55:18                    43   
408865  2019-06-30 20:54:26 2019-06-30 21:07:56                   115   
408866  2019-06-30 20:54:10 2019-06-30 21:07:41                   115   
411348  2019-06-30 19:06:44 2019-06-30 19:31:03                   140   
...                     ...                 ...                   ...   
3187013 2019-10-01 07:59:02 2019-10-01 08:17:23                    78   
3187038 2019-10-01 07:57:34 2019-10-01 08:06:52                   112   
3187124 2019-10-01 07:51:42 2019-10-01 08:18:20                   216   
3187608 2019-10-01 07:17:24 2019-10-01 07:43:40                    36   
3187830 2019-10-01 06:42:26 2019-10-01 19:03:31                   228   

         Return station id  
407705               204.0  
408025               204.0  
408865               204.0  
408866 